# TP4b - Kolmogorov Complexity

In [5]:
import numpy as np
import pandas as pd
import gzip
import pickle
import math
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In this exercise, you will explore how compression algorithms can be used to approximate the Kolmogorov complexity of datasets and learning tasks.



Kolmogorov complexity $K(x)$ of an object $x$ (typically a string) is the length of the shortest possible program that can produce that object $x$ as output. Unfortunately it is an uncomputable quantity in the general case, but we can approximate it using practical compression tools such as zip. The compressors exploit patterns, redundancy, and structure in data — just as machine learning models do.

This connection leads to a fascinating interpretation: learning can be viewed as compression. In both unsupervised and supervised learning, the goal is to describe data as compactly as possible, either by identifying structure in the data itself or by predicting labels from features.

In [4]:
# Load dataset
data = load_diabetes()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Upper bounds of the Kolmogorov complexity $K$ can be obtained by considering various encoders (=compressors). Given an encoder $h$, and data $x$ to compress, the bound on $K(x)$ is given by the size of the compressed data $h(x)$ + the size of the decoder $h^{-1}$, since one needs to describe both of them to re-generate the original data. 

Indeed, given data $x$ to encode (e.g. whole Wikipedia), imagine an encoder that associates that particular $x$ to just one bit, and applies zip otherwise (i.e. to any other string). The encoding of $x$ will be very short, but it is just because $x$ is actually stored in the decoder.

Moreover, in the case of lossy encoders, i.e. that are not lossless (e.g., jpg encoder for images, or machine learning models in general), then one has to take into account that the decoder $h^{-1}$ will not produce the original data $x$ but a corrupted version $x' = h^{-1} \circ h(x)$ (note the slight abuse of notation: $h^{-1}$ is the decoder, not exactly matching the encoder $h$ as the latter is not injective, so in general $h^{-1} \circ h \neq Id$). Therefore, one needs in plus to describe the residuals $z = x'-x$, in order to be able to generate the original data *exactly*. Given the decoded reconstruction $x'$, describing $z$ or $x$ is equivalent. The associated description cost is $K(z|x') = K(z|h^{-1}, h(x)) = K(x|h^{-1}, h(x))$.

As a result, in the general case we have:
$$ K(x) \leq K(h^{-1}) + K( h(x) ) + K(x|h^{-1}, h(x))$$
that is, the cost of describing the decoder, the cost of describing the encoded data (which is upper bounded by its length + twice its $\log$), and the cost of describing residuals if the encoder is lossy.


Note that Kolmogorov complexity does not include any computational complexity consideration (all programs, whether fast or very slow, are treated equally).



In `unsupervised learning`, we are primarily interested in describing the structure of the data itself, such as clustering or density estimation. The goal is to find the shortest description of the dataset $x$. Considering lossless compression only, there is no residuals to describe. Thus, we have for instance:
$$ K(x) \leq |\text{zip}(x)| + |\text{unzip program}|$$

In `supervised learning`, we do not only describe the inputs $x$, but also learn a predictor, i.e. a mapping from $x$ to the labels $y$, which makes mistakes. Here, the goal is to describe $y$ given $x$, using a model $h$. We will consider that for any sample $x_i$, the model outputs a distribution of probability $p_h(y|x_i)$ over possible labels. As the inputs $x$ are given to the predictor, they do not need to be encoded, and in the Kolmogorov complexity, only two terms remain: the description of the model (seen as a decoder) and the description of the true labels $y$ given the predictions $p_h(y|x_i)$ (similarly to residuals):
$$ K(y|x) \leq K(h) + K(y|h,x) $$
where $K(y|h,x)$ stands for the log likelihood error of our model (over the whole dataset) given the data $x$, that is, $\sum_i -\log p_h(y_i|x_i)$, which is precisely the number of bits of information one should give to identify, for each input sample $x_i$, the true label $y_i$ given the prediction $p_h(y_i|x_i)$ of the model.

This machine learning instanciation of the Kolmogorov complexity is named the _Minimum Description Length (MDL)_ paradigm.

The Kolmogorov complexity $K(h)$ of the model itself can be approximated using the Bayesian Information Criterion (BIC) or the Akaike Information Criterion (AIC), based on the number of parameters of the models (and possibly the number of samples).

## Practical
1. Use `gzip` to compute a Kolmogorov complexity upper bound, assuming an unsupervised task on the dataset features. The data should be turned into binary first using pickle.

2. Train the models given on the dataset.

3. Use the trained models to compute Kolmogorov complexity upper bounds in the supervised setting. For MDL you should consider both the Bayesian and Akaike criterions. It is up to you to find the correct number of parameters for its model.

4. How do the upper bounds compare between the different supervised models? Explain your answer.

In [6]:
# Initialize Models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Support Vector Machine": SVR(),
    "k-Nearest Neighbors": KNeighborsRegressor()
}

# Function to compute compressed size as Kolmogorov complexity approximation
def compute_compressed_size(obj):
    pickled = pickle.dumps(obj, protocol=pickle.HIGHEST_PROTOCOL)
    compressed = gzip.compress(pickled)
    return len(compressed) * 8  # convert bytes → bits

# TODO: compute Kolmogorov complextiy upper bounds
def safe_nll_bits(y_true, y_pred, eps: float = 1e-8):
    mse = mean_squared_error(y_true, y_pred)
    sigma2 = max(mse, eps)
    n = len(y_true)
    nll_nat = n / 2 * (math.log(2 * math.pi * sigma2) + 1)
    return nll_nat / math.log(2)  # nat → bits


def num_parameters(model, name: str, X_sample):
    if name == "Linear Regression":
        return model.coef_.size + 1  # β plus intercept
    if name == "Decision Tree":
        return model.tree_.node_count # each node stores a threshold & feature index
    if name == "Support Vector Machine":
        return (model.support_vectors_.size +
                model.dual_coef_.size + 1)
    if name == "k-Nearest Neighbors":
        return X_sample.size
    raise ValueError(f"Unknown model type: {name}")

compressed_X_bits = compute_compressed_size(X_train)
compressed_xy_bits = compute_compressed_size((X_train, y_train))

results = []  # to store everything for a small recap table

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)

    nll_bits = safe_nll_bits(y_train, y_pred_train)
    k = num_parameters(model, name, X_train)
    n = len(X_train)

    nll_nat = nll_bits * math.log(2)

    aic_nat = 2 * k + 2 * nll_nat
    bic_nat = k * math.log(n) + 2 * nll_nat

    aic_bits = aic_nat / math.log(2)
    bic_bits = bic_nat / math.log(2)

    K_xy_AIC = compressed_X_bits + aic_bits
    K_xy_BIC = compressed_X_bits + bic_bits

    results.append(
        (name, k, nll_bits, aic_bits, bic_bits,
         K_xy_AIC, K_xy_BIC)
    )

header = ("Model", "k (params)", "−log₂L  (bits)",
          "K(h) AIC  (bits)", "K(h) BIC  (bits)",
          "K(x,y) AIC (bits)", "K(x,y) BIC (bits)")
print("{:>23s} | {:>10s} | {:>15s} | {:>15s} | {:>15s} | {:>15s} | {:>15s}".format(*header))
print("-"*120)
for row in results:
    print("{:>23s} | {:10d} | {:15.1f} | {:15.1f} | {:15.1f} | {:15.1f} | {:15.1f}".format(*row))

print("\nUnsupervised gzip bound on K(x)            : {:,.0f} bits".format(compressed_X_bits))
print("Unsupervised gzip bound on K(x,y) (baseline): {:,.0f} bits".format(compressed_xy_bits))

                  Model | k (params) |  −log₂L  (bits) | K(h) AIC  (bits) | K(h) BIC  (bits) | K(x,y) AIC (bits) | K(x,y) BIC (bits)
------------------------------------------------------------------------------------------------------------------------
      Linear Regression |         11 |          2749.9 |          5531.6 |          5592.9 |        105795.6 |        105856.9
          Decision Tree |        691 |         -3967.9 |         -5942.1 |         -2087.6 |         94321.9 |         98176.4
 Support Vector Machine |       3873 |          2894.6 |         16964.3 |         38568.4 |        117228.3 |        138832.4
    k-Nearest Neighbors |       3530 |          2717.8 |         15621.0 |         35311.8 |        115885.0 |        135575.8

Unsupervised gzip bound on K(x)            : 100,264 bits
Unsupervised gzip bound on K(x,y) (baseline): 107,688 bits


Remember that in the supervised case we ommit the complexity of the data itself. This is because we are not interested in compressing the data, but rather in compressing the labels given the data. However, in order to have a fair comparisson with the unsupervised case, we will add this complexity to the mix. Thus, the consider upper bounds on the joint distribution $(x,y)$ instead of $y|x$:
$$ K(x,y) \leq K(h) + K(x) + K(y|h,x)$$
and we approximate $K(x)$ by $|\text{zip}(x)| $.

5. Recompute the upper bound in the supervised case using the new formula.

6. Now that the results are comparable, how do the upper bounds behave between the supervised and the unsupervised setting?

7. Why is the zip compression so bad when dealing with tabular data?

8. What are the assumptions behind using AIC and BIC for estimating the description length of a model? What types of models or settings might violate these assumptions?

9. In the supervised setting, why is it important to separate the complexity of the model from the complexity of the residuals? What does this decomposition tell us ?

**Answers:**

**5. Recompute the upper bound in the supervised case using the new formula.**  
The joint‑MDL bound  
$$
K(x,y)\;\le\;K(h)\;+\;K(x)\;+\;K(y\,|\,h,x)
$$  
adds the gzip description of the features, **K(x)= 100,264 bits**, to each model’s two‑part AIC/BIC length.  
The numbers after this addition are:

| Model | **K(x,y) AIC** | **K(x,y) BIC** |
|-------|----------------|----------------|
| Linear Regression | **105,796 bits** | **105,857 bits** |
| Decision Tree | **94,322 bits** | **98,176 bits** |
| Support Vector Machine | **117,228 bits** | **138,832 bits** |
| k‑Nearest Neighbors | **115,885 bits** | **135,576 bits** |

(The unsupervised baselines remain: gzip K(x)= 100,264 bits, gzip K(x,y)= 107,688 bits.)

---

**6. Now that the results are comparable, how do the upper bounds behave between the supervised and the unsupervised setting?**  
* **Decision Tree** beats both unsupervised benchmarks by a large margin (≈ 13 kbits [AIC] or ≈ 9 kbits [BIC]), proving that a model‑plus‑residual code can compress better than generic zip.  
* **Linear Regression** performs slightly better (≈ 1.8 kbits) than gzip K(x,y).  
* **k‑NN** and **SVR** are worse than gzip because their massive decoders add more bits than they save in residuals.  
Hence learning helps only when the model is expressive and compact enough to overcome its own description cost.

---

**7. Why is the zip compression so bad when dealing with tabular data?**  
* **Binary noise.** Floats differ in low‑order bits; identical byte substrings—what gzip needs—are rare.  
* **Structure ignorance.** gzip sees a flat stream, not rows and columns; correlations across columns/rows are invisible.  
* **No semantic transforms.** It cannot delta‑code, quantise, or exploit linear relations; specialised columnar codecs or statistical models can.

---

**8. What are the assumptions behind using AIC and BIC for estimating the description length of a model? What types of models or settings might violate these assumptions?**  

| Criterion | Key assumptions | Violations / risky settings |
|-----------|-----------------|-----------------------------|
| **AIC** | i.i.d. data, large‑sample asymptotics, candidate models near the truth | Heavy regularisation, small‑n data, autocorrelated time‑series, over‑parameterised deep nets |
| **BIC** | All AIC assumptions plus "true model is in the set" and fixed finite parameter count k | Non‑parametric/lazy models (k‑NN), kernel/GP methods, trees whose node count grows with n, deep networks |

When these premises fail, the penalties (2*k* for AIC, *k* log n for BIC) cease to represent a true description‑length contribution.

---

**9. In the supervised setting, why is it important to separate the complexity of the model from the complexity of the residuals? What does this decomposition tell us ?**  
* **Interpretation.** *K(h)* measures how elaborate the rule is; *K(y | h,x)* measures the information the rule still misses.  
* **Over‑fit control.** A larger model is worthwhile only if it cuts more bits from the residual message than it adds to K(h).  
* **Bias–variance view.** K(h) ↔ capacity/bias, residual code ↔ unexplained variance/noise.  
* **MDL principle.** The best hypothesis minimises the sum, aligning statistical learning with optimal lossless compression.